In [9]:
import pickle as pkl
import numpy as np
import pandas as pd
import operator
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.metrics import log_loss, roc_auc_score, f1_score
from xgboost import XGBClassifier

In [50]:
def f1_score_single(y_true, y_pred):
    y_true = set(list(y_true)) if len(y_true) > 0 else set([-1])
    y_pred = set(list(y_pred))
    cross_size = len(y_true & y_pred)
    if cross_size == 0: return 0.
    p = 1. * cross_size / len(y_pred)
    r = 1. * cross_size / len(y_true)
    return 2 * p * r / (p + r)
    
def f1_score(y_true, y_pred):
    return np.mean([f1_score_single(x, y) for x, y in zip(y_true, y_pred)])

In [15]:
X = pd.read_csv("../data/reorder_features_train.csv", index_col=0)
X.head()

/home/joe/kaggle/instacart/.env/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,product_id,user_id,product_count,in_last_order1,in_last_order2,in_last_order3,total_orders,order_number,order_dow,order_hour_of_day,days_since_prior_order,dummy,order_id,aisle_id,department_id
0,196,1,10,True,True,True,10,11,4,8,14.0,1,1187899,77,7
1,12427,1,10,True,True,True,10,11,4,8,14.0,1,1187899,23,19
2,10258,1,9,True,True,True,10,11,4,8,14.0,1,1187899,117,19
3,25133,1,8,True,True,True,10,11,4,8,14.0,1,1187899,21,16
4,46149,1,3,True,True,True,10,11,4,8,14.0,1,1187899,77,7


In [12]:
reorders = pd.read_csv("../data/reorder_preds_train.csv", index_col=0)
none_preds = pd.read_csv("../data/none_preds_train.csv", index_col=0)

/home/joe/kaggle/instacart/.env/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [21]:
reorders.index = X["order_id"]
reorders["product_id"] = X["product_id"].values

In [23]:
reorders.head()

,xgb,linear,mean,product_id
order_id,,,,
1187899,0.598291,0.820084,0.709188,196
1187899,0.598291,0.800088,0.699189,12427
1187899,0.598291,0.763885,0.681088,10258
1187899,0.598291,0.723740,0.661016,25133
1187899,0.341717,0.481698,0.411708,46149


In [44]:
truth = pd.read_csv('../data/order_products__train.csv', dtype=np.int32, index_col=0)
truth = truth.groupby("order_id")[["product_id", "reordered"]].apply(lambda x: list(x.loc[x["reordered"] == True, "product_id"]))
truth.shape

/home/joe/kaggle/instacart/.env/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(131209,)

In [ ]:
for thr in np.linspace(0.1, 0.2, 3):
    for thr2 in np.linspace(0.1, 0.5, 3):
        preds = reorders[reorders["mean"] >= thr].groupby(level=0)["product_id"].apply(list).loc[truth.index]
        nulls = preds[(preds.isnull())|(none_preds["mean"] > thr2)].index
        for i in nulls:
            preds.loc[i] = [-1]
        score = f1_score(truth.values, preds.values)
        print(thr, thr2, score)

0.1 0.1 0.318160960413
0.1 0.3 0.335695359146
0.1 0.5 0.329815721577
0.15 0.1 0.342589695114
0.15 0.3 0.365846811077
